# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-05-02&until-occurred-date=2021-05-09


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

12276 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-02&until-occurred-date=2021-05-09
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=e7415cb9-f356-4ded-81c3-7f255c65a946&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-02&until-occurred-date=2021-05-09
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=d51aecbd-a689-4545-897c-0686c1f641f5&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-02&until-occurred-date=2021-05-09
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=d621a098-9f03-451a-960e-3e6178ac1e8a&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load .DS_Store
failed to load .gitkeep
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.05.06.21256403...
REST API query complete  200
REST API query started for 10.1101/2021.04.10.21255248...
REST API query complete  200
REST API query started for 10.1101/2021.02.18.21251986...
REST API query complete  200
REST API query started for 10.1101/2021.02.27.433180...
REST API query complete  200
REST API query started for 10.1101/2020.04.30.066209...
REST API query complete  200
REST API query started for 10.1101/2021.04.29.441939...
REST API query complete  200
REST API query started for 10.1101/2021.03.11.21253225...
REST API query complete  200
REST API query started for 10.1101/2021.04.27.441510...
REST API query complete  200
REST API query started for 10.1101/2020.09.10.20191619...
REST API query complete  200
REST API query started for 10.1101/2021.01.29.21250653...
REST API query complete  200
REST API query started for 10.1101/2020.08.21.20179671...
REST API query complete  200
REST API query started for 10.1101/2021.03.11.21253

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.05.06.21256403,1607,medRxiv,Infectious Diseases (except HIV/AIDS),True,Detection of persistent SARS-CoV-2 IgG antibod...,"[{'name': 'Aubree Mades'}, {'name': 'Prithivi ...",<p>Previous studies have shown that mRNA COVID...,2021-05-07
1,10.1101/2021.05.02.442312,133,bioRxiv,Microbiology,False,Aminoglycoside antibiotics inhibit phage infec...,"[{'name': 'Larissa Kever'}, {'name': 'Aël Hard...","<p>In response to viral predation, bacteria ha...",2021-05-02
2,10.1101/2021.05.01.442252,65,bioRxiv,Cancer Biology,False,Mesenchymal Lineage Heterogeneity Underlies No...,"[{'name': 'Erin J. Helms'}, {'name': 'Mark W. ...",<p>Cancer-associated fibroblast (CAF) heteroge...,2021-05-02
3,10.1101/2021.05.02.442311,53,bioRxiv,Microbiology,False,Vibrio cholerae biofilm dispersal regulator ca...,"[{'name': 'Praveen K. Singh'}, {'name': 'Danie...",<p>The extracellular matrix is a defining feat...,2021-05-02
4,10.1101/2021.05.06.442916,45,bioRxiv,Microbiology,True,Identification of DAXX As A Restriction Factor...,"[{'name': 'Alice Mac Kain'}, {'name': 'Ghizlan...",<p>While interferon restricts SARS-CoV-2 repli...,2021-05-06
5,10.1101/2021.05.01.441648,41,bioRxiv,Neuroscience,False,A multisensory circuit for gating intense aver...,"[{'name': 'Arun Asok'}, {'name': 'Félix Leroy'...",<p>The ventral hippocampus (vHPC) is critical ...,2021-05-02
6,10.1101/2021.05.02.442342,41,bioRxiv,Molecular Biology,False,Cytosolic aggregation of mitochondrial protein...,"[{'name': 'Urszula Nowicka'}, {'name': 'Piotr ...",<p>Mitochondria are organelles with their own ...,2021-05-02
7,10.1101/2021.05.01.442281,40,bioRxiv,Bioinformatics,False,RNA splicing programs define tissue compartmen...,"[{'name': 'Julia Eve Olivieri'}, {'name': 'Roo...",<p>More than 95% of human genes are alternativ...,2021-05-02
8,10.1101/2021.05.03.442461,29,bioRxiv,Plant Biology,False,Misregulation of MYB16 causes stomatal cluster...,"[{'name': 'Shao-Li Yang'}, {'name': 'Ngan Tran...",<p>Stomata and leaf cuticle regulate water eva...,2021-05-03
9,10.1101/2021.05.04.442674,28,bioRxiv,Neuroscience,False,Functional and ultrastructural analysis of rea...,"[{'name': 'Iris Odstrcil'}, {'name': 'Mariela ...",<p>All animals need to differentiate between e...,2021-05-05


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

57 preprints (including 14 covering SARS-CoV-2, 46 from bioRxiv and 11 from medRxiv) published in the last 7 days before 2021-05-09 had been tweeted at least 5 times (maximum 1607).